# 4.4.4. Regresión polinomial
Ahora podemos explorar estos conceptos de forma interactiva ajustando polinomios a los datos.

In [21]:
use strict;
use warnings;
use Data::Dump qw(dump);
use Data::Dumper qw(Dumper);
use AI::MXNet qw(mx);
use List::Util qw(max min shuffle);
#use PDL qw(pdl);
use d2l;
use d2l::Timer;
use d2l::Animator;
use d2l::Accumulator;
IPerl->load_plugin('Chart::Plotly');
use Chart::Plotly::Plot;

## 4.4.4.1. Generación del conjunto de datos
Primero necesitamos datos. Dado x , usaremos el siguiente polinomio cúbico para generar las etiquetas en los datos de entrenamiento y prueba:

(4.4.2)
y=5+1.2x−3.4x22!+5.6x33!+ϵ donde ϵ∼N(0,0.12).
 
El término de ruido ϵ sigue una distribución normal con una media de 0 y una desviación estándar de 0,1. Para la optimización, normalmente queremos evitar valores muy grandes de gradientes o pérdidas. ¡Esta es la razón por la cual las características se reescalan de xi a xii! . Nos permite evitar valores muy grandes para exponentes i grandes. Sintetizaremos 100 muestras cada una para el conjunto de entrenamiento y el conjunto de prueba.

In [70]:
my $max_degree = 20;  # Maximum degree of the polynomialmy 
my $n_train= 100;
my $ n_test = 100;  # Training and test dataset sizes
my @true_w = mx->nd($max_degree);  # Allocate lots of empty space
@true_w[0..4] = mx->nd->array([5, 1.2, -3.4, 5.6]);
my $features = nd->random->normal(0,1 ,[$n_train + $n_test, 1]);
shuffle($features);
my @poly_features = nd->power($features, nd->arange($max_degree)->reshape(1, -1));

for (my $i->range(@$max_degree)){

    @poly_features[i] /= mx->nd(i + 1);  # `gamma(n)` = (n-1)!
# Shape of `labels`: (`n_train` + `n_test`,)
    my $labels = nd->dot($poly_features, $true_w);
    $labels += nd->random->normal(scale=0.1, size=$labels->shape);
}

Error: Bareword "i" not allowed while "strict subs" in use at reply input line 12.
BEGIN not safe after errors--compilation aborted at reply input line 20.



Nuevamente, los monomios almacenados en poly_features son reescalados por la función gamma, donde Γ(n)=(n−1)! . Eche un vistazo a las primeras 2 muestras del conjunto de datos generado. El valor 1 es técnicamente una característica, es decir, la característica constante correspondiente al sesgo.

In [76]:
print dump $features[:2], $poly_features[:2, :], $labels[:2];

Error: syntax error at reply input line 1, near "[:"
Global symbol "@poly_features" requires explicit package name (did you forget to declare "my @poly_features"?) at reply input line 1.
Global symbol "@labels" requires explicit package name (did you forget to declare "my @labels"?) at reply input line 1.
BEGIN not safe after errors--compilation aborted at reply input line 5.



## 4.4.4.2. Entrenamiento y prueba del modelo
Primero implementemos una función para evaluar la pérdida en un conjunto de datos dado.

In [19]:
sub evaluate_loss {
  my $net;
  my $data_iter;
  my $loss;
  my $metric = d2l->Accumulator(2);  # Sum of losses, no. of examples
    for my $X, $y (@$data_iter){
        my $l = $loss($net($X), $y);
        my $metric($l->sum(), $l->size)
   }
    return $metric[0] / $metric[1]
}

Warning: "my" variable $loss masks earlier declaration in same scope at reply input line 7.

"my" variable $net masks earlier declaration in same scope at reply input line 7.

"my" variable $X masks earlier declaration in same statement at reply input line 7.

"my" variable $metric masks earlier declaration in same scope at reply input line 8.


Error: syntax error at reply input line 6, near "$X,"
Global symbol "$y" requires explicit package name (did you forget to declare "my $y"?) at reply input line 6.
Can't redeclare "my" in "my" at reply input line 8, near ";
        my"
syntax error at reply input line 8, near "$metric("
Global symbol "$l" requires explicit package name (did you forget to declare "my $l"?) at reply input line 8.
Global symbol "$l" requires explicit package name (did you forget to declare "my $l"?) at reply input line 8.
Global symbol "@metric" requires explicit package name (did you forget to declare "my @metric"?) at reply input line 10.
Global symbol "@metric" requires explicit package name (did you forget to declare "my @metric"?) at reply input line 10.
BEGIN not safe after errors--compilation aborted at reply input line 15.



Ahora defina la función de entrenamiento.

In [ ]:
sub train my ($train_features, $test_features, $train_labels, $test_labels,
          $num_epochs=400):
    my $loss = gluon.loss.L2Loss();
    my $net = nn.Sequential();
    # Switch off the bias since we already catered for it in the polynomial
    # features
    my $net.add(nn.Dense(1, my $use_bias=False));
    my $net.initialize()
    my $batch_size = min(10, $train_labels->shape[0])
    my $train_iter = d2l->load_array(($train_features, $train_labels), $batch_size)
    $test_iter = d2l->load_array(($test_features, $test_labels), $batch_size,
                               is_train=False)
    my $trainer = gluon->Trainer($net->collect_params(), 'sgd',
                            {'learning_rate': 0.01})
    my $animator = d2l->Animator(my $xlabel='epoch', my $ylabel='loss', my $yscale='log',
                            my $xlim=[1, my $num_epochs], my $ylim=[1e-3, 1e2],
                            my $legend=['train', 'test']);
    for epoch in range(num_epochs):
        d2l.train_epoch_ch3(net, train_iter, loss, trainer)
        if epoch == 0 or (epoch + 1) % 20 == 0:
            animator.add(epoch + 1, (evaluate_loss(
                net, train_iter, loss), evaluate_loss(net, test_iter, loss)))
    print('weight:', net[0].weight.data().asnumpy())

## 4.4.4.3. Ajuste de función polinomial de tercer orden (normal)
Comenzaremos usando primero una función polinomial de tercer orden, que es del mismo orden que la función de generación de datos. Los resultados muestran que las pérdidas de entrenamiento y prueba de este modelo pueden reducirse de manera efectiva. Los parámetros del modelo aprendido también están cerca de los valores reales w=[5,1.2,−3.4,5.6] .

In [ ]:
ck the first four dimensions, i.e., 1, x, x^2/2!, x^3/3! from the
# polynomial features
train(poly_features[:n_train, :4], poly_features[n_train:, :4],
      labels[:n_train], labels[n_train:])

## 4.4.4.4. Ajuste de función lineal (ajuste inferior)
Echemos otro vistazo al ajuste de funciones lineales. Después del declive en las primeras épocas, se vuelve difícil disminuir aún más la pérdida de entrenamiento de este modelo. Una vez completada la última iteración de época, la pérdida de entrenamiento sigue siendo alta. Cuando se utilizan para ajustar patrones no lineales (como aquí la función polinomial de tercer orden), los modelos lineales tienden a no ajustarse correctamente.


train(poly_features[:n_train, :2], poly_features[n_train:, :2],
      labels[:n_train], labels[n_train:]);

## 4.4.4.5. Ajuste de funciones polinómicas de orden superior (sobreajuste)
Ahora intentemos entrenar el modelo usando un polinomio de grado demasiado alto. Aquí, no hay datos suficientes para saber que los coeficientes de mayor grado deberían tener valores cercanos a cero. Como resultado, nuestro modelo excesivamente complejo es tan susceptible que está siendo influenciado por el ruido en los datos de entrenamiento. Aunque la pérdida de entrenamiento se puede reducir de manera efectiva, la pérdida de prueba sigue siendo mucho mayor. Muestra que el modelo complejo sobreajusta los datos.

In [ ]:
train(poly_features[:n_train, :], poly_features[n_train:, :],
      labels[:n_train], labels[n_train:], num_epochs=1500);

En las secciones subsiguientes, continuaremos analizando los problemas de sobreajuste y los métodos para tratarlos, como la disminución del peso y la deserción.

## 4.4.5. Resumen
Dado que el error de generalización no se puede estimar en función del error de entrenamiento, simplemente minimizar el error de entrenamiento no significará necesariamente una reducción en el error de generalización. Los modelos de aprendizaje automático deben tener cuidado de protegerse contra el sobreajuste para minimizar el error de generalización.

Se puede usar un conjunto de validación para la selección del modelo, siempre que no se use con demasiada liberalidad.

Underfitting significa que un modelo no es capaz de reducir el error de entrenamiento. Cuando el error de entrenamiento es mucho menor que el error de validación, hay sobreajuste.

Deberíamos elegir un modelo adecuadamente complejo y evitar usar muestras de entrenamiento insuficientes.